In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd

## Lets go, Keras!

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Activation, InputLayer
from keras.layers.core import Dense, Flatten, Dropout

In [3]:
x_train = pd.read_csv("Athens_x_train.csv")
y_train = pd.read_csv("Athens_y_train.csv")

x_test = pd.read_csv("Athens_x_test.csv")
y_test = pd.read_csv("Athens_y_test.csv")

In [4]:
def drop_samevalue_columns(df):
    for col in range(len(df.columns)-1, -1, -1):
        row_list = []
        column_arr = df.iloc[:, col].to_list()
        for row in range(len(df)):
            row_list.append(column_arr[row])
        
        if len(set(row_list)) == 1:
            df.pop(df.columns[col])
    return

In [5]:
# drop columns which have only 1 value accross all rows
drop_samevalue_columns(x_train)
drop_samevalue_columns(x_test)

# apply normalization techniques
for column in x_train.columns:
	x_train[column] = (x_train[column] - x_train[column].mean()) / x_train[column].std()
	
for column in x_test.columns:
	x_test[column] = (x_test[column] - x_test[column].mean()) / x_test[column].std()

In [6]:
x_train

,N_cx,cx_0_1,cx_0_2,cx_0_3,cx_0_4,cx_1_0,cx_1_2,cx_1_3,cx_1_4,cx_2_0,...,readout_error_1,T1_2,T2_2,readout_error_2,T1_3,T2_3,readout_error_3,T1_4,T2_4,readout_error_4
0,-0.083980,0.845916,-0.676863,-0.681556,-0.698119,-0.678601,0.835610,0.840605,-0.685110,-0.679708,...,2.191481,-0.493675,-0.454840,-0.276212,0.283728,-0.390606,-0.145888,-0.708391,-1.936209,0.136675
1,-0.083980,0.845916,-0.676863,-0.681556,-0.698119,2.354311,0.835610,-0.686509,0.074916,0.843436,...,-0.243231,-2.394010,-2.027583,1.150873,1.339800,-0.312660,-0.523907,0.949129,0.635158,-0.272392
2,-0.687137,0.845916,-0.676863,-0.681556,-0.698119,0.837855,-0.682108,-0.686509,-0.685110,-0.679708,...,-0.090395,0.994013,0.248109,-0.240937,-2.905421,-0.367970,1.471594,-1.864955,-2.563778,-0.105046
3,-0.342475,0.845916,0.842848,0.854977,0.780893,-0.678601,-0.682108,-0.686509,-0.685110,-0.679708,...,-0.342752,1.852855,-0.549416,-0.376668,2.839530,0.008133,-0.343984,-1.280889,-0.968885,-0.300283
4,-0.600971,-0.672293,-0.676863,-0.681556,-0.698119,-0.678601,-0.682108,-0.686509,-0.685110,-0.679708,...,-0.165036,0.845608,0.521856,-0.243238,0.598689,2.173392,-0.158610,-0.124530,0.171161,-0.170125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21401,-0.859467,-0.672293,-0.676863,-0.681556,-0.698119,-0.678601,-0.682108,0.840605,-0.685110,0.081864,...,-0.059591,-0.955807,-0.401426,-0.199528,-0.940484,-0.831174,-0.418498,0.158788,0.627831,-0.173224
21402,0.691508,-0.672293,0.842848,0.854977,2.259906,1.596083,0.076751,0.840605,0.834942,-0.679708,...,0.204614,0.094344,0.230440,-0.351362,-1.001424,-0.335706,-0.409411,1.266196,1.054317,-0.225907
21403,-0.687137,-0.672293,-0.676863,-0.681556,0.780893,-0.678601,-0.682108,-0.686509,-0.685110,-0.679708,...,-0.271665,0.120190,0.800982,-0.194927,0.764986,-0.900382,-0.147706,0.054855,1.272595,-0.210412
21404,-0.600971,0.086811,-0.676863,-0.681556,-0.698119,0.837855,-0.682108,-0.686509,-0.685110,-0.679708,...,-0.271665,0.120190,0.800982,-0.194927,0.764986,-0.900382,-0.147706,0.054855,1.272595,-0.210412


### Defining a one-hot-encoding function

In [7]:
def oneHotEncodeDF(df):
  for qbit in range(5):
    for oneHotValues in range(5):
      # We make a column for each of the 5 one_hot values, per qubit
      column = []
      for row in range(len(df)):
        if df[str(qbit)][row]==oneHotValues:
          column.append(1)
        else:
          column.append(0)

      # Add the column to the dataframe
      df[str(qbit)+"_"+str(oneHotValues)] = column

  # Droping the initial five columns
  df.drop(['0', '1', '2', '3', '4'], axis=1, inplace=True)


### Defining a Repair Operator

In [8]:
# Input list has length 5x5 = 25

# This function uses the repair operator approach
def repair_operator(arr):
  assert len(arr)==25

  final_array = [-1, -1, -1, -1 ,-1]

  # Splitting the list
  q0 = arr[0:5]
  q1 = arr[5:10]
  q2 = arr[10:15]
  q3 = arr[15:20]
  q4 = arr[20:25]

  # List of lists
  original = [q0, q1, q2, q3, q4]
  list_partitions = [q0, q1, q2, q3, q4]

  # Each iteration in while decides an output qbit for one of the 5 input qbits
  while(len(list_partitions)!=0):
    maxim = -1
    for iter in list_partitions:
      maxim=max(maxim, max(iter))

    for given_list in list_partitions:
      if maxim == max(given_list):
        pos = given_list.index(maxim)
        final_array[original.index(given_list)] = pos

        # removing that list and the possibility of another list getting the same qubit index
        list_partitions.remove(given_list)
        for iter in list_partitions:
          iter[pos] = -1
        break

  return final_array

In [9]:
def get_final_outputs(output):
    ans = output.tolist()
    new = []
    for arr in ans:
        new.append(repair_operator(arr))
    return new

def get_accuracy(inp, labl):
    inputs = inp.values.tolist()
    labels = labl.values.tolist()
    correct = 0
    total = 0
    for i in range(len(inputs)):
        if inputs[i][0]==labels[i][0] and inputs[i][1]==labels[i][1] and inputs[i][2]==labels[i][2] and inputs[i][3]==labels[i][3] and inputs[i][4]==labels[i][4]:
            correct+=1
        total+=1
    acc = (correct*100)/total
    print("Total Predictions : ", total)
    print("Correct Predictions : ", correct)
    print("Accuracy : ", round(acc, 5))
    return

In [10]:
# Testing the repair operator
foo = [0.15, 0.23, 0.12 ,0.06, 0.4, 0.08, 0.16, 0.25, 0.13, 0.2, 0.05, 0.06, 0.31, 0.33, 0.2, 0.45, 0.1, 0.03, 0.15, 0.17, 0.27, 0.2, 0.18, 0.06, 0.18]
print(repair_operator(foo))
# output: [4, 2, 3, 0, 1]

[4, 2, 3, 0, 1]


## Final NN

In [96]:
# Data elements have 78 features

# Output layer has 25 neurons (5x5)
# for each of the 5 circuit qbits, we have 5 probabilities corresponding to the 5 processor qbits
'''
nL0 = Sequential([
    Dense(264, input_shape=(52,), activation='relu'),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    #Dense(1024, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])'''

nL0 = Sequential([
    Dense(256, input_shape=(52,), activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])

nL1 = Sequential([
    Dense(50, input_shape=(52,), activation='relu'),
    Dropout(0.2),
    Dense(24, activation='relu'),
    Dense(5, activation='softmax')
])

nL2 = Sequential([
    Dense(264, input_shape=(52,), activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])

nL3 = Sequential([
    Dense(264, input_shape=(52,), activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])

nL4 = Sequential([
    Dense(264, input_shape=(52,), activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])

In [97]:
# One hot encoding the label vector and feeding the data into the neural network
training_labels = y_train.copy()

oneHotEncodeDF(training_labels)

# Five different training lables for the 5 different NNs on the same input dataset
train_0 = training_labels.iloc[:, 0:5]
train_1 = training_labels.iloc[:, 5:10]
train_2 = training_labels.iloc[:, 10:15]
train_3 = training_labels.iloc[:, 15:20]
train_4 = training_labels.iloc[:, 20:25]

In [98]:
# Compiling the neural Layouts
optim = keras.optimizers.Adam(learning_rate=0.03)

neuralLayouts = [nL0, nL1, nL2, nL3, nL4]
for nL in neuralLayouts:
    nL.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

nL0.summary()

Model: "sequential_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_337 (Dense)           (None, 256)               13568     
                                                                 
 dense_338 (Dense)           (None, 512)               131584    
                                                                 
 dropout_106 (Dropout)       (None, 512)               0         
                                                                 
 dense_339 (Dense)           (None, 1024)              525312    
                                                                 
 dropout_107 (Dropout)       (None, 1024)              0         
                                                                 
 dense_340 (Dense)           (None, 256)               262400    
                                                                 
 dropout_108 (Dropout)       (None, 256)             

In [99]:
# Neural Net 0
nL0.fit(x_train, train_0, validation_split=0.2, batch_size=50, epochs=150, verbose=1)

Epoch 1/150
343/343 [==============================] - 3s 7ms/step - loss: 1.5257 - accuracy: 0.3525 - val_loss: 1.4992 - val_accuracy: 0.3557
Epoch 2/150
343/343 [==============================] - 2s 6ms/step - loss: 1.5018 - accuracy: 0.3551 - val_loss: 1.5018 - val_accuracy: 0.3578
Epoch 3/150
343/343 [==============================] - 2s 6ms/step - loss: 1.4913 - accuracy: 0.3573 - val_loss: 1.5075 - val_accuracy: 0.3494
Epoch 4/150
343/343 [==============================] - 2s 6ms/step - loss: 1.4815 - accuracy: 0.3607 - val_loss: 1.4961 - val_accuracy: 0.3594
Epoch 5/150
343/343 [==============================] - 2s 6ms/step - loss: 1.4698 - accuracy: 0.3608 - val_loss: 1.5019 - val_accuracy: 0.3554
Epoch 6/150
343/343 [==============================] - 2s 6ms/step - loss: 1.4559 - accuracy: 0.3710 - val_loss: 1.4999 - val_accuracy: 0.3559
Epoch 7/150
343/343 [==============================] - 2s 6ms/step - loss: 1.4405 - accuracy: 0.3811 - val_loss: 1.5084 - val_accuracy: 0.3435

In [71]:
# Neural Net 1
nL1.fit(x_train, train_1, validation_split=0.2, batch_size=50, epochs=50, verbose=1)

Epoch 1/50
343/343 [==============================] - 1s 1ms/step - loss: 1.5770 - accuracy: 0.3298 - val_loss: 1.5259 - val_accuracy: 0.3484
Epoch 2/50
343/343 [==============================] - 0s 1ms/step - loss: 1.5190 - accuracy: 0.3610 - val_loss: 1.5167 - val_accuracy: 0.3533
Epoch 3/50
343/343 [==============================] - 0s 1ms/step - loss: 1.5073 - accuracy: 0.3621 - val_loss: 1.5182 - val_accuracy: 0.3519
Epoch 4/50
343/343 [==============================] - 0s 965us/step - loss: 1.5014 - accuracy: 0.3650 - val_loss: 1.5135 - val_accuracy: 0.3547
Epoch 5/50
343/343 [==============================] - 0s 1ms/step - loss: 1.4962 - accuracy: 0.3667 - val_loss: 1.5129 - val_accuracy: 0.3550
Epoch 6/50
343/343 [==============================] - 0s 975us/step - loss: 1.4923 - accuracy: 0.3682 - val_loss: 1.5107 - val_accuracy: 0.3573
Epoch 7/50
343/343 [==============================] - 0s 970us/step - loss: 1.4876 - accuracy: 0.3712 - val_loss: 1.5117 - val_accuracy: 0.3536


In [48]:
# Neural Net 2
nL2.fit(x_train, train_2, validation_split=0.2, batch_size=50, epochs=50, verbose=1)

Epoch 1/3
343/343 [==============================] - 3s 6ms/step - loss: 1.5066 - accuracy: 0.3662 - val_loss: 1.4911 - val_accuracy: 0.3657
Epoch 2/3
343/343 [==============================] - 2s 6ms/step - loss: 1.4840 - accuracy: 0.3690 - val_loss: 1.4759 - val_accuracy: 0.3660
Epoch 3/3
343/343 [==============================] - 2s 6ms/step - loss: 1.4724 - accuracy: 0.3715 - val_loss: 1.4789 - val_accuracy: 0.3641


In [49]:
# Neural Net 3
nL3.fit(x_train, train_3, validation_split=0.2, batch_size=50, epochs=50, verbose=1)

Epoch 1/3
343/343 [==============================] - 2s 6ms/step - loss: 1.5196 - accuracy: 0.3636 - val_loss: 1.5022 - val_accuracy: 0.3746
Epoch 2/3
343/343 [==============================] - 2s 6ms/step - loss: 1.4943 - accuracy: 0.3700 - val_loss: 1.4941 - val_accuracy: 0.3786
Epoch 3/3
343/343 [==============================] - 2s 6ms/step - loss: 1.4820 - accuracy: 0.3732 - val_loss: 1.4910 - val_accuracy: 0.3774


In [50]:
# Neural Net 4
nL4.fit(x_train, train_4, validation_split=0.2, batch_size=50, epochs=50, verbose=1)

Epoch 1/3
343/343 [==============================] - 2s 6ms/step - loss: 1.5332 - accuracy: 0.3506 - val_loss: 1.5121 - val_accuracy: 0.3610
Epoch 2/3
343/343 [==============================] - 2s 6ms/step - loss: 1.5089 - accuracy: 0.3527 - val_loss: 1.5104 - val_accuracy: 0.3620
Epoch 3/3
343/343 [==============================] - 2s 6ms/step - loss: 1.4959 - accuracy: 0.3595 - val_loss: 1.5101 - val_accuracy: 0.3557


In [51]:
# *****************************************************

In [52]:
pred0 = pd.DataFrame(nL0.predict(x_test))
pred1 = pd.DataFrame(nL1.predict(x_test))
pred2 = pd.DataFrame(nL2.predict(x_test))
pred3 = pd.DataFrame(nL3.predict(x_test))
pred4 = pd.DataFrame(nL4.predict(x_test))

prediction = pd.concat([pred0, pred1, pred2, pred3, pred4], axis=1, ignore_index=True)

168/168 [==============================] - 0s 2ms/step


In [53]:
prediction

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.178387,0.135483,0.142969,0.162730,0.380432,0.158530,0.166376,0.150685,0.354266,0.170143,...,0.115332,0.503176,0.124654,0.152377,0.104460,0.300512,0.145695,0.144742,0.173618,0.235432
1,0.196486,0.153939,0.183496,0.132668,0.333412,0.198270,0.178424,0.143764,0.355302,0.124240,...,0.208036,0.316814,0.163580,0.167383,0.144187,0.316975,0.159380,0.143385,0.160853,0.219407
2,0.254425,0.108058,0.086498,0.137158,0.413860,0.137794,0.175203,0.156339,0.355791,0.174872,...,0.160585,0.295321,0.204627,0.181869,0.157599,0.282257,0.222028,0.165205,0.184543,0.145967
3,0.190643,0.112256,0.101211,0.166487,0.429403,0.066421,0.114384,0.126271,0.604442,0.088482,...,0.153760,0.432252,0.149663,0.143000,0.121325,0.346149,0.234873,0.202446,0.156057,0.060475
4,0.192511,0.110332,0.070600,0.111026,0.515532,0.131236,0.147182,0.048139,0.616097,0.057347,...,0.054425,0.600482,0.101592,0.156655,0.086847,0.669514,0.042522,0.050686,0.078401,0.158877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5347,0.309751,0.104564,0.228474,0.121599,0.235612,0.171495,0.158780,0.202566,0.284156,0.183003,...,0.188426,0.264971,0.192032,0.188334,0.166238,0.303657,0.169525,0.151793,0.155894,0.219131
5348,0.251319,0.128285,0.194928,0.133331,0.292136,0.150738,0.190859,0.178783,0.310296,0.169324,...,0.211249,0.267377,0.176681,0.185719,0.158974,0.393012,0.202728,0.171122,0.153875,0.079263
5349,0.238509,0.153411,0.128827,0.129718,0.349534,0.162426,0.117146,0.123308,0.394890,0.202231,...,0.072482,0.460595,0.194271,0.168914,0.103737,0.376773,0.134493,0.128579,0.156271,0.203885
5350,0.190001,0.079601,0.050178,0.090924,0.589296,0.302786,0.165289,0.060266,0.411603,0.060056,...,0.115014,0.425775,0.165499,0.177542,0.116170,0.340479,0.163399,0.146455,0.176824,0.172843


In [54]:
prediction = pd.DataFrame(get_final_outputs(np.array(prediction)))

In [55]:
prediction

,0,1,2,3,4
0,4,3,2,1,0
1,4,3,2,1,0
2,4,3,2,1,0
3,4,3,2,1,0
4,4,3,2,1,0
...,...,...,...,...,...
5347,0,3,2,1,4
5348,4,3,2,1,0
5349,4,3,2,1,0
5350,4,3,2,1,0


In [56]:
y_test

,0,1,2,3,4
0,4.0,0.0,1.0,2.0,3.0
1,4.0,3.0,2.0,1.0,0.0
2,3.0,2.0,4.0,0.0,1.0
3,0.0,1.0,2.0,3.0,4.0
4,4.0,3.0,2.0,1.0,0.0
...,...,...,...,...,...
5347,2.0,3.0,1.0,4.0,0.0
5348,4.0,1.0,0.0,3.0,2.0
5349,4.0,3.0,2.0,1.0,0.0
5350,4.0,3.0,2.0,1.0,0.0


In [57]:
get_accuracy(prediction, y_test)

Total Predictions :  5352
Correct Predictions :  1075
Accuracy :  20.08595


In [58]:
# *****************************************************